<a href="https://colab.research.google.com/github/speakerv/Testing/blob/master/Copy_of_m5_SIA_model_v35.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pdb
import os
import gc
import warnings

import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

from scipy.optimize import minimize
from IPython.display import Audio

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)
register_matplotlib_converters()
sns.set()

%xmode Verbose
# %pdb on

Exception reporting mode: Verbose


In [0]:
import requests, zipfile, io
#The copied URL goes here ->
r = requests.get( 'https://github.com/speakerv/Testing/blob/master/Achive%20for%20M5%20part%201.zip?raw=true' ) 
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

r = requests.get( 'https://github.com/speakerv/Testing/blob/master/Achive%20for%20M5%20part%202.zip?raw=true' ) 
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [0]:
import IPython

def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

In [0]:
def on_kaggle():
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ

In [0]:
if on_kaggle():
    os.system("pip install --quiet mlflow_extend")

In [0]:
def reduce_mem_usage(df, verbose=False):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    int_columns = df.select_dtypes(include=["int"]).columns
    float_columns = df.select_dtypes(include=["float"]).columns

    for col in int_columns:
        df[col] = pd.to_numeric(df[col], downcast="integer")

    for col in float_columns:
        df[col] = pd.to_numeric(df[col], downcast="float")

    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [0]:
def read_data():
    INPUT_DIR_1 = "/content/Achive for M5 part 1"
    INPUT_DIR_2 = "/content/Achive for M5 part 2"
    print("Reading files...")

    calendar = pd.read_csv(f"{INPUT_DIR_1}/calendar.csv").pipe(reduce_mem_usage)
    prices = pd.read_csv(f"{INPUT_DIR_2}/sell_prices.csv").pipe(reduce_mem_usage)

    sales = pd.read_csv(f"{INPUT_DIR_1}/sales_train_validation.csv",).pipe(
        reduce_mem_usage)
        
    submission = pd.read_csv(f"{INPUT_DIR_1}/sample_submission.csv").pipe(
        reduce_mem_usage)
    

    print("sales shape:", sales.shape)
    print("prices shape:", prices.shape)
    print("calendar shape:", calendar.shape)
    print("submission shape:", submission.shape)

    # calendar shape: (1969, 14)
    # sell_prices shape: (6841121, 4)
    # sales_train_val shape: (30490, 1919)
    # submission shape: (60980, 29)    
    return sales, prices, calendar, submission

In [10]:
try:
  sales
except:
  sales, prices, calendar, submission = read_data()

Reading files...
sales shape: (30490, 1919)
prices shape: (6841121, 4)
calendar shape: (1969, 14)
submission shape: (60980, 29)


In [11]:
df_sales=sales.drop(["item_id", "dept_id", "cat_id", "store_id", "state_id"], axis=1)
df_sales.set_index(["id"],inplace=True)
new_index=np.arange(1,len(df_sales)+1).tolist()
df_sales=df_sales.transpose()
df_sales=df_sales.reset_index()
df_sales=df_sales.rename(columns={'index':'d'})
df_sales['wday']=calendar['wday'][0:1913].values
df_sales


id,d,HOBBIES_1_001_CA_1_validation,HOBBIES_1_002_CA_1_validation,HOBBIES_1_003_CA_1_validation,HOBBIES_1_004_CA_1_validation,HOBBIES_1_005_CA_1_validation,HOBBIES_1_006_CA_1_validation,HOBBIES_1_007_CA_1_validation,HOBBIES_1_008_CA_1_validation,HOBBIES_1_009_CA_1_validation,HOBBIES_1_010_CA_1_validation,HOBBIES_1_011_CA_1_validation,HOBBIES_1_012_CA_1_validation,HOBBIES_1_013_CA_1_validation,HOBBIES_1_014_CA_1_validation,HOBBIES_1_015_CA_1_validation,HOBBIES_1_016_CA_1_validation,HOBBIES_1_017_CA_1_validation,HOBBIES_1_018_CA_1_validation,HOBBIES_1_019_CA_1_validation,HOBBIES_1_020_CA_1_validation,HOBBIES_1_021_CA_1_validation,HOBBIES_1_022_CA_1_validation,HOBBIES_1_023_CA_1_validation,HOBBIES_1_024_CA_1_validation,HOBBIES_1_025_CA_1_validation,HOBBIES_1_026_CA_1_validation,HOBBIES_1_027_CA_1_validation,HOBBIES_1_028_CA_1_validation,HOBBIES_1_029_CA_1_validation,HOBBIES_1_030_CA_1_validation,HOBBIES_1_031_CA_1_validation,HOBBIES_1_032_CA_1_validation,HOBBIES_1_033_CA_1_validation,HOBBIES_1_034_CA_1_validation,HOBBIES_1_035_CA_1_validation,HOBBIES_1_036_CA_1_validation,HOBBIES_1_037_CA_1_validation,HOBBIES_1_038_CA_1_validation,HOBBIES_1_039_CA_1_validation,HOBBIES_1_040_CA_1_validation,HOBBIES_1_041_CA_1_validation,HOBBIES_1_042_CA_1_validation,HOBBIES_1_043_CA_1_validation,HOBBIES_1_044_CA_1_validation,HOBBIES_1_045_CA_1_validation,HOBBIES_1_046_CA_1_validation,HOBBIES_1_047_CA_1_validation,HOBBIES_1_048_CA_1_validation,HOBBIES_1_049_CA_1_validation,HOBBIES_1_050_CA_1_validation,HOBBIES_1_051_CA_1_validation,HOBBIES_1_052_CA_1_validation,HOBBIES_1_053_CA_1_validation,HOBBIES_1_054_CA_1_validation,HOBBIES_1_055_CA_1_validation,HOBBIES_1_056_CA_1_validation,HOBBIES_1_057_CA_1_validation,HOBBIES_1_058_CA_1_validation,HOBBIES_1_060_CA_1_validation,HOBBIES_1_061_CA_1_validation,HOBBIES_1_062_CA_1_validation,HOBBIES_1_063_CA_1_validation,HOBBIES_1_064_CA_1_validation,HOBBIES_1_065_CA_1_validation,HOBBIES_1_066_CA_1_validation,HOBBIES_1_067_CA_1_validation,HOBBIES_1_068_CA_1_validation,HOBBIES_1_069_CA_1_validation,HOBBIES_1_070_CA_1_validation,HOBBIES_1_072_CA_1_validation,HOBBIES_1_073_CA_1_validation,HOBBIES_1_074_CA_1_validation,HOBBIES_1_075_CA_1_validation,HOBBIES_1_076_CA_1_validation,HOBBIES_1_077_CA_1_validation,HOBBIES_1_078_CA_1_validation,HOBBIES_1_079_CA_1_validation,HOBBIES_1_080_CA_1_validation,HOBBIES_1_081_CA_1_validation,HOBBIES_1_082_CA_1_validation,HOBBIES_1_083_CA_1_validation,HOBBIES_1_084_CA_1_validation,HOBBIES_1_085_CA_1_validation,HOBBIES_1_086_CA_1_validation,HOBBIES_1_087_CA_1_validation,HOBBIES_1_088_CA_1_validation,HOBBIES_1_089_CA_1_validation,HOBBIES_1_090_CA_1_validation,HOBBIES_1_091_CA_1_validation,HOBBIES_1_092_CA_1_validation,HOBBIES_1_093_CA_1_validation,HOBBIES_1_094_CA_1_validation,HOBBIES_1_095_CA_1_validation,HOBBIES_1_097_CA_1_validation,HOBBIES_1_098_CA_1_validation,HOBBIES_1_099_CA_1_validation,HOBBIES_1_100_CA_1_validation,HOBBIES_1_102_CA_1_validation,HOBBIES_1_103_CA_1_validation,HOBBIES_1_104_CA_1_validation,HOBBIES_1_105_CA_1_validation,HOBBIES_1_106_CA_1_validation,HOBBIES_1_107_CA_1_validation,HOBBIES_1_108_CA_1_validation,HOBBIES_1_109_CA_1_validation,HOBBIES_1_110_CA_1_validation,HOBBIES_1_111_CA_1_validation,HOBBIES_1_112_CA_1_validation,HOBBIES_1_113_CA_1_validation,HOBBIES_1_114_CA_1_validation,HOBBIES_1_115_CA_1_validation,HOBBIES_1_116_CA_1_validation,HOBBIES_1_117_CA_1_validation,HOBBIES_1_118_CA_1_validation,HOBBIES_1_119_CA_1_validation,HOBBIES_1_120_CA_1_validation,HOBBIES_1_121_CA_1_validation,HOBBIES_1_122_CA_1_validation,HOBBIES_1_123_CA_1_validation,HOBBIES_1_124_CA_1_validation,HOBBIES_1_125_CA_1_validation,HOBBIES_1_126_CA_1_validation,HOBBIES_1_127_CA_1_validation,HOBBIES_1_128_CA_1_validation,HOBBIES_1_129_CA_1_validation,HOBBIES_1_130_CA_1_validation,HOBBIES_1_131_CA_1_validation,HOBBIES_1_132_CA_1_validation,HOBBIES_1_133_CA_1_validation,HOBBIES_1_134_CA_1_validation,HOBBIES_1_135_CA_1_validation,HOBBIES_1_136_CA_1_validation,HOBBIES_1_137_CA_1_validation,HOBBI

In [12]:
item=15300
mean_for_item_per_day_of_week=np.empty(7,dtype=float)
k_day_of_week=np.empty(7,dtype=float)
for i in range(7):
  mean_for_item_per_day_of_week[i]=df_sales.loc[df_sales['wday'] == i+1][df_sales.columns[item+1]].mean()
for i in range(6):
  k_day_of_week[i]=mean_for_item_per_day_of_week[i+1]/mean_for_item_per_day_of_week[i]

k_day_of_week[6]=mean_for_item_per_day_of_week[0]/mean_for_item_per_day_of_week[6]

print(mean_for_item_per_day_of_week)
print(k_day_of_week)

train_data_item=df_sales.iloc[:,item+1]
print(train_data_item)

[1.75182482 1.46350365 1.22710623 1.16849817 1.23809524 1.27838828
 1.46520147]
[0.83541667 0.83847159 0.95223881 1.05956113 1.03254438 1.14613181
 1.19562044]
0       0
1       0
2       0
3       0
4       0
       ..
1908    2
1909    4
1910    2
1911    0
1912    0
Name: HOBBIES_1_056_TX_2_validation, Length: 1913, dtype: int16


In [0]:
# train_data=train_data_item.to_frame()
# train_data.columns=['Value']
# train_data=pd.read_csv("/content/test1_train_data.csv")
# train_data['Value']
# train_data

In [0]:
# INPUT_DIR = "output/kaggle/working"
# train_data=sales.loc[0][6:].to_frame()
# train_data.columns=['Value']
# train_data
# train_data.to_csv("test1_train_data.csv",index=True)

# train_data=pd.read_csv("/content/test1_train_data.csv")
# train_data['Value']

train_data=train_data_item.to_frame()
train_data.columns=['Value']

fig = go.Figure()
# x = np.arange(len(prediction))
fig.add_trace(go.Scatter(x=train_data.index.values,
                         y=train_data['Value'],
                         name ='pre_interval')) 
fig.show()

num_for_zeros_window=10
counter_zeros_in_sequence = 0 
flag_start_count=False
for i in range(len(train_data)):
      
    if train_data['Value'][i] != 0 and flag_start_count:
        flag_start_count = False 
        if counter_zeros_in_sequence >= num_for_zeros_window:
            # print('point 4')           
            train_data=train_data.drop(index=np.arange(left_index,right_index).tolist())                   
                       
    
    if train_data['Value'][i] == 0 and flag_start_count:
        # print('point 2')
        counter_zeros_in_sequence += 1
        right_index=i
        
    
    if train_data['Value'][i] == 0 and not(flag_start_count):
        # print('point 1')
        flag_start_count = True
        counter_zeros_in_sequence = 1
        left_index=i

fig = go.Figure()
# x = np.arange(len(prediction))
fig.add_trace(go.Scatter(x=train_data.index.values,
                         y=train_data['Value'],
                         name ='pre_interval')) 
    
    

In [0]:
# Making random not only in order?
def generate_random_batch(series, size=DAYS_PRED):      
    
    left_non_zero_index=0
    right_index=series.shape[0]-2*size    
    
    while series.iloc[left_non_zero_index][0]==0:
        left_non_zero_index +=1

    r_i=np.random.randint(left_non_zero_index,right_index)
    # r_i=600 #for overfiting traingin

    batch_1=series.iloc[r_i:(r_i+size)]
    batch_2=series.iloc[r_i+size:r_i+size*2]
    
    return batch_1, batch_2, r_i

# batch_1, batch_2, r_i=generate_random_batch(train_data)
# print(batch_1)


In [0]:
def validation(model,display=False,display_figure=False):
#     prediction_2=[]
    y=[]
    summation=[]
#   for i in range(1300,1800,20):
#     for i in np.random.randint(train_data.index):  #need correction!!!
    for _ in range(1):
        i = np.random.randint(train_data.index.values.shape[0]-DAYS_PRED*2)  #need correction!!!  
        # i=600 #for overfiting traingin
        batch_test_1, batch_test_2, batch_true, df_batch_true = test_batch_ri_from_train_data(train_data, r_i=i)

        noise_batch_test_1=noise_detection(batch_test_1)
        noise_batch_test_2=noise_detection(batch_test_2)
        prediction=f28_days_prediction(model,noise_batch_test_1,noise_batch_test_2,df_batch_2)+tf.reduce_mean(batch_test_1).numpy()

        # prediction=np.zeros([28,])+tf.reduce_mean(batch_test_1).numpy()
        for i in range(len(prediction)):
          if (prediction[i])<0: prediction[i]=0

                
        # prediction=f28_days_prediction(model,batch_test_1,batch_test_2,df_batch_2)              
              
                
        y.append(rmsse(np.reshape(batch_true,28),prediction, display))
        summation.append(sum_delta(np.reshape(batch_true,28),prediction))   
    if display_figure:    
        fig = go.Figure()
        x = np.arange(len(y),len(prediction)+DAYS_PRED)
        fig.add_trace(go.Scatter(x=x, y=y, name ='tatata'))
        fig.show()
    if display:    
        print(f" summation: {sum(summation)}")
        print(f" RMSE: {sum(y)}")
    return sum(summation), sum(y)
   

def batch_transformation(batch):
    batch = tf.transpose(np.expand_dims(batch,axis=0))       
    batch_t=np.copy(batch)
    return batch_t
    
#need correction!!!
def optimization(model, batch_1,batch_2,df_batch_true,i_day,display=False):

    # batch_t1=batch_transformation(batch_1)
    # batch_t2=batch_transformation(batch_2)
       
    fun_value=np.empty(shape=[1,],dtype = float)
    # max_index = tf.math.argmax(batch_1,axis=1)    
    # tf_max_value = batch_1[0,np.asscalar(max_index.numpy()),0]

    # #max_value = np.asscalar(tf_max_value.numpy())
    
    # for i in range(0,tf_max_value):  
    #                         #need correction!!!
    #     batch_2[0,27,0]=i
    #     fun_value=np.append(fun_value,model([batch_1, batch_2]).numpy())

    max_index = tf.math.argmax(tf.math.abs(batch_1),axis=1)
    tf_max_value = tf.math.round(tf.math.abs(batch_1[0,np.asscalar(max_index.numpy()),0] )).numpy().astype(int)    
    for i in range(-tf_max_value-1,tf_max_value+1):                           
        batch_2[0,27,0]=i
        fun_value=np.append(fun_value,model([batch_1, batch_2]).numpy())  
  
    fun_value=np.delete(fun_value,0,axis=0)
    i_min=np.argmin(fun_value)

    # print(f"i_min:{i_min}, batch_1[i_min]: {batch_1.index[i_min]}")

    #add feature_1:wday
    k_i=calendar.iloc[df_batch_true.index[i_day]]['wday']
    # optimized_value=fun_value[i_min]*k_day_of_week[k_i-1]
    optimized_pridection=i_min-1-tf_max_value
    # optimized_pridection=round(i_min*k_day_of_week[k_i-1])
    # print(f"k_i: {k_i}, k_day_of_week[k_i-1]: {k_day_of_week[k_i-1]}")
    # optimized_value=0

    # print(f" function values: {fun_value} \n \
    #     optimezid value {optimized_pridection}\
    #     min index (or index + 1 element): {i_min}  \n \
    #     function min value: {fun_value[i_min]}")
    
    if display:
        print(f" function values: {fun_value} \n \
        optimezid value {optimized_pridection}\
        min index (or index + 1 element): {i_min}  \n \
        function min value: {fun_value[i_min]}")
        
    return fun_value, optimized_pridection
# _ , i_min  = optimization(model,batch_1,batch_2,display=True)
# print(i_min)

def f28_days_prediction(model,batch_1,batch_2,df_batch_true):

    # batch_t1=batch_transformation(batch_1)
    # batch_t2=batch_transformation(batch_2)

    prediction=np.empty(shape=[1,],dtype = float)
    for i_day in range(28):    
        _, new_prediction = optimization(model,batch_1,batch_2,df_batch_true,i_day,display=False)
        prediction=np.append(prediction, new_prediction)
        batch_2[0,27,0]=new_prediction
        batch_1 = np.copy(batch_2) 
        batch_2 = np.roll(batch_2,-1)    

    prediction=np.delete(prediction,0,axis=0)
    return prediction

# prediction=f28_days_prediction(batch_1,batch_2)
# print(prediction)
# print(prediction.shape,type(prediction))
# fig = go.Figure()
# x = np.arange(len(prediction))
# fig.add_trace(go.Scatter(x=x, y=prediction))
# fig.show()

def test_batch_ri_from_train_data(series, size=DAYS_PRED,r_i = DAYS_PRED):      
    
    left_non_zero_index=0
#   right_index=series.shape[0]-2*size    
    right_index=r_i
    
    while series.iloc[left_non_zero_index][0]==0:
        left_non_zero_index +=1
        
    if right_index<left_non_zero_index: right_index=left_non_zero_index   
       
    df_batch_1=series.iloc[r_i:(r_i+size)]
    df_batch_2=series.iloc[r_i:(r_i+size)]
    df_batch_true=series.iloc[(r_i+size+1):(r_i+1+size*2)]
 
    batch_1 = tf.transpose(np.expand_dims(df_batch_1,axis=0))
    batch_2 = tf.transpose(np.expand_dims(df_batch_2,axis=0))
    batch_true = tf.transpose(np.expand_dims(df_batch_true,axis=0))
    
    # batch_1 = np.expand_dims(batch_1,axis=0)
    # batch_2 = np.expand_dims(batch_2,axis=0)
    # batch_true = np.expand_dims(batch_true,axis=0)
    
    batch_2 = np.roll(batch_2,-1)
    
    return batch_1, batch_2, batch_true, df_batch_true

# batch_1, batch_2, batch_true = test_batch_ri_from_train_data(train_data, size=DAYS_PRED,r_i = DAYS_PRED) 
# print(batch_2)

def rmsse(y_true, y_pred,display=False):
    if display:
        print(f"y_true sum: {np.sum(y_true)}, y_pred sum: {np.sum(y_pred)}")
    return np.sqrt(mean_squared_error(y_true, y_pred))

def sum_delta(y_true, y_pred):    
    return abs(np.sum(y_true)-np.sum(y_pred))


In [0]:
class Layer_MSE(layers.Layer):
    global model

    def __init__(self, units=32):
        super(Layer_MSE, self).__init__()
        
        
    def call(self, input1, input2):
#         print(input1.shape)
        self.out=tf.math.add(input1,-input2)
#         print(f"delta:{out}")
        self.out=self.out*self.out
#         print(f"out*out:{out}")
        self.out=tf.math.reduce_sum(self.out,axis=2)
#         print(f"reduce_sum:{out}")        
        self.out = tf.math.sqrt(self.out)        
#       print(out)
        return self.out

# x1 = tf.constant([[[1.0, 1.0], [1.0, 1.0]],
#                    [[1.0, 1.0], [1.0, 1.0]],
#                    [[1.0, 1.0], [1.0, 1.0]]])
# x2 =  tf.constant([[[1.0, 2.0], [3.0, 4.0]],
#                    [[1.0, 2.0], [3.0, 4.0]],
#                    [[1.0, 2.0], [3.0, 4.0]]])
# print(x1.shape)

# test_layer = Layer_MSE()
# y = test_layer(x1,x2)
# print(y)

In [0]:
# loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# tf.keras.losses.Reduction.NONE

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = myloss(G_model, inputs, targets, training=True) # True or False?
        # print(f"loss_value in grad fucntion: {loss_value}")
    return loss_value, tape.gradient(loss_value, G_model.trainable_variables)

def training_loop():
    # Note: Rerunning this cell uses the same model variables
    global TRAIN_STAGE
    global BATCH_SIZE
    global G_model
   
    # Keep results for plotting
    train_loss_results = []
    train_accuracy_results = []
    
    optimizer = tf.keras.optimizers.Adadelta()
    print(optimizer)
    shift_for_train_stage=0
    for epoch in range(NUM_EPOCH):
        epoch_loss_avg = tf.keras.metrics.Mean()
        # epoch_accuracy = tf.keras.metrics.Accuracy()
        epoch_accuracy = tf.keras.metrics.MeanAbsoluteError()

      # Training loop     
#       for x,y in [[x_train, y_train]]:
        for _ in range(BATCH_SIZE):
            i=np.random.randint(len(y_train))            
            # x = [np.expand_dims(x_train[0][i],axis=0), np.expand_dims(x_train[0][1],axis=0)]
            # pdb.set_trace()
            x = [np.expand_dims(x_train[0][0][i*28:(i+1)*28],axis=0), np.expand_dims(x_train[1][0][i*28:(i+1)*28],axis=0)]
            y = np.expand_dims(y_train[i],axis=0)
            
        # Optimize the model
            loss_value, grads = grad(G_model, x, y)
            # print(f"x value in training loop : {x}")
            # print(f"y value in training loop : {y}")
            # print(f"grads value in training loop : {grads}")
            optimizer.apply_gradients(zip(grads, G_model.trainable_variables))

            # Track progress
            epoch_loss_avg.update_state(loss_value)  # Add current batch loss
            # Compare predicted label to actual label
            # training=True is needed only if there are layers with different
            # behavior during training versus inference (e.g. Dropout).
            epoch_accuracy.update_state(y, G_model(x, training=True))

        # End epoch
        train_loss_results.append(epoch_loss_avg.result())
        train_accuracy_results.append(epoch_accuracy.result())

        #Save the model
        if (epoch % 10) == 0:
          save_path = r'/content/Saved models'
          name_of_file ="SIA_model_v20"
          complete_path = os.path.join(save_path,name_of_file)
          G_model.save(complete_path)
          print("Model Saved")

        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}, TRAIN_STAGE: {}".format(epoch,
                                                                    epoch_loss_avg.result(),
                                                                    epoch_accuracy.result(),
                                                                    TRAIN_STAGE))
        if (epoch-shift_for_train_stage)>10 and np.all(np.array(train_loss_results)[-10:]<=EARLY_STOP_LOSS_VALUE):
           print(f"------ New TRAIN_STAGE---------")
           TRAIN_STAGE +=1
           shift_for_train_stage=epoch
          #  BATCH_SIZE = 2
          #  optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.1)
           optimizer = tf.keras.optimizers.Adam()
           print(optimizer)
          

        if TRAIN_STAGE == 3:
            break
       
#         if epoch % 50 == 0:
#             print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
#                                                                     epoch_loss_avg.result(),
#                                                                     epoch_accuracy.result()))


# optimizer = tf.keras.optimizers.Adam()

# loss_value, grads = grad(model, features, labels)

# print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
#                                           loss_value.numpy()))

# optimizer.apply_gradients(zip(grads, model.trainable_variables))

# print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
#                                           loss(model, features, labels, training=True).numpy()))

In [0]:
def create_model_v2(model_name=''):
    inputs_1 = keras.Input(shape=(DAYS_PRED,1))
    inputs_2 = keras.Input(shape=(DAYS_PRED,1))

    list_convlayer=[]
    list_kernel_size=[]
    for kernel_size_i in range(3,10):
      list_convlayer.append( layers.Conv1D(NUM_FILTERS_IN_CONV_LAYER,
                                  kernel_size_i,
                                  padding='valid',
                                  activation='selu',
                                  kernel_initializer=tf.random_uniform_initializer(0,1,seed=70),                                                        
                                ))
      list_kernel_size.append(kernel_size_i)

    list_x1=[]
    list_x2=[]
    for convlayer in list_convlayer:
      list_x1.append(convlayer(inputs_1))
      list_x2.append(convlayer(inputs_2))

    list_out=[]
    denselayer=layers.Dense(len(list_kernel_size), kernel_initializer=tf.ones_initializer())
    
    for i in range(len(list_x1)):
      list_x1[i]=layers.LayerNormalization()(list_x1[i])
      list_x2[i]=layers.LayerNormalization()(list_x2[i])
      list_x1[i]=layers.AveragePooling1D(pool_size=DAYS_PRED-list_kernel_size[i]+1)(list_x1[i])
      list_x2[i]=layers.AveragePooling1D(pool_size=DAYS_PRED-list_kernel_size[i]+1)(list_x2[i])
      # list_x1[i]=denselayer(list_x1[i])
      # list_x2[i]=denselayer(list_x2[i])
      # list_x1[i]=layers.Dense(len(list_kernel_size))(list_x1[i])
      # list_x2[i]=layers.Dense(len(list_kernel_size))(list_x2[i])
      list_out.append(Layer_MSE()(list_x1[i],list_x2[i]))
      
     
    out=tf.reduce_sum(list_out,0)
    model = keras.Model([inputs_1,inputs_2], [out], name=model_name)
            
    return model

In [0]:
def create_model_v3(model_name=''):

    inputs_1 = keras.Input(shape=(DAYS_PRED,1))
    inputs_2 = keras.Input(shape=(DAYS_PRED,1))

    denselayer1=layers.Dense(DAYS_PRED,
                            kernel_initializer=tf.random_uniform_initializer(0,1,seed=70),
                            activation='tanh'
                            )
    x_1=denselayer1(inputs_1)
    x_2=denselayer1(inputs_2)
    
    x_1 = layers.LayerNormalization()(x_1)
    x_2 = layers.LayerNormalization()(x_2) 

    convlayer = layers.Conv1D(NUM_FILTERS_IN_CONV_LAYER,
                              KERNEL_SIZE,
                              padding='valid',
                              activation='linear',
                              kernel_initializer=tf.random_uniform_initializer(0,1,seed=70),                                                        
                             )

    x_1 = convlayer(x_1)
    x_2 = convlayer(x_2)
  
    x_1 = layers.AveragePooling1D(pool_size=DAYS_PRED-KERNEL_SIZE+1)(x_1)
    x_2 = layers.AveragePooling1D(pool_size=DAYS_PRED-KERNEL_SIZE+1)(x_2)

    denselayer2=layers.Dense(DAYS_PRED-KERNEL_SIZE+1,
                            kernel_initializer=tf.random_uniform_initializer(0,1,seed=70),
                            activation='sigmoid'
                            )
    x_1=denselayer2(x_1)
    x_2=denselayer2(x_2)
     
    out=Layer_MSE()(x_1,x_2)
   
    model = keras.Model([inputs_1,inputs_2], [out], name=model_name)
        
    return model

In [0]:
def create_model(model_name=''):
    inputs_1 = keras.Input(shape=(DAYS_PRED,1))
    inputs_2 = keras.Input(shape=(DAYS_PRED,1))
#     BN = layers.BatchNormalization()
#     x_1 = BN(inputs_1)
#     x_2 = BN(inputs_2)    
    convlayer = layers.Conv1D(NUM_FILTERS_IN_CONV_LAYER,
                              KERNEL_SIZE,
                              padding='valid',
                              activation=ACTIVATION_FUNCTION,
                              kernel_initializer=tf.random_uniform_initializer(0,1,seed=70),                                                        
                             )
#     x_1 = convlayer(x_1)
#     x_2 = convlayer(x_2)  
    x_1 = convlayer(inputs_1)
    x_2 = convlayer(inputs_2)
    # x_1 = layers.LayerNormalization()(x_1)
    # x_2 = layers.LayerNormalization()(x_2) 
    # denselayer=layers.Dense(NUM_FILTERS_IN_CONV_LAYER, kernel_initializer=tf.ones_initializer())
    # x_1=denselayer(x_1)
    # x_2=denselayer(x_2)
    x_1 = layers.AveragePooling1D(pool_size=DAYS_PRED-KERNEL_SIZE+1)(x_1)
    x_2 = layers.AveragePooling1D(pool_size=DAYS_PRED-KERNEL_SIZE+1)(x_2)

     

    out=Layer_MSE()(x_1,x_2)         
#     out=layers.Reshape(())(out)
   
    model = keras.Model([inputs_1,inputs_2], [out], name=model_name)
        
    return model
 
def train_model():
    global G_model
    # model = create_model()
    G_model = create_model_v3()

    keras.utils.plot_model(G_model,show_shapes=True)
        
    # training_loop(model)
    training_loop()
    
    # return model


In [0]:
# sales, prices, calendar, submission = read_data()
# NUM_ITEMS = sales.shape[0]  # 30490
# DAYS_PRED = submission.shape[1] - 1  # 28

# Glogbal Parameters
NUM_ITEMS =  30490
DAYS_PRED =  28

# Global tuning variables
NUM_FILTERS_IN_CONV_LAYER=7
KERNEL_SIZE=14
NUM_EPOCH = 3000
BATCH_SIZE = 5
NUM_SAMPLES = 100
EARLY_STOP_LOSS_VALUE=1
ACTIVATION_FUNCTION = 'tanh'

TRAIN_STAGE=0

@tf.function 
def tffun(x):
  y=tf.constant([[x]])  
  return y

def myloss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  # y_ = model(x, training=training)
        
    if TRAIN_STAGE <-5:
      y_ = model(x, training=True)
      RMSE=0
      summation=0
      return abs(y_)
    else:
      y_ = model(x, training=False)  
      summation, RMSE = validation(model)
      print(f"in myloss function: summation {summation}, RMSE: {RMSE}, TRAIN_STAGE: {TRAIN_STAGE}")
      # pdb.set_trace()
    return RMSE*10+abs(y_)      
      

def noise_detection(batch): 
   if tf.is_tensor(batch):     
     return batch-tf.reduce_mean(batch).numpy()
   else:
     return batch-batch.mean()

In [0]:
# bb=tf.Tensor([[2]])
# dd=tf.Tensor( shape=(1, 1), dtype=float32, numpy=array([[0.07228721]]))
dd=tf.Tensor([[2.0]],dtype=float32,value_index=)
dd

In [0]:
test1_model=create_model_v3("test1 model")
test1_model.summary()
keras.utils.plot_model(test1_model,show_shapes=True)
# print(model.trainable_variables)   

In [194]:
# START CALCULATION
# %run -d
# pdb.set_trace()

try:
  del G_model
except:
  print("model dont't exist")

try:
  del model
except:
  print("model dont't exist")

input_1=np.empty(shape=[28,1],dtype = float)
input_2=np.empty(shape=[28,1],dtype = float)
for i in range(NUM_SAMPLES):
    df_batch_1, df_batch_2, _ = generate_random_batch(train_data)

    df_batch_1 = noise_detection(df_batch_1)
    df_batch_2 = noise_detection(df_batch_2)
#     batch_1, _, _ = generate_random_batch(train_data)
#     _, batch_2, _ = generate_random_batch(train_data)

    # batch_1 = tf.transpose(np.expand_dims(df_batch_1,axis=0))
    # batch_2 = tf.transpose(np.expand_dims(df_batch_2,axis=0))

    # batch_1 = np.expand_dims(batch_1,axis=0)
    # batch_2 = np.expand_dims(batch_2,axis=0)   
    input_1=np.append(input_1,df_batch_1,axis=0)
    input_2=np.append(input_2,df_batch_2,axis=0)

input_1=np.delete(input_1,0,axis=0)
input_2=np.delete(input_2,0,axis=0)

y_train=pd.Series(np.zeros(NUM_SAMPLES),dtype = float)
input_1=np.expand_dims(input_1,axis=0)
input_2=np.expand_dims(input_2,axis=0)
x_train = [input_1, input_2]

TRAIN_STAGE=1
# model=train_model()
train_model()
keras.utils.plot_model(G_model,show_shapes=True)

Audio(filename="/content/Achive for M5 part 1/ring_end.mp3", autoplay=True)

model dont't exist
in myloss function: summation 94.0, RMSE: 7.358183005529092, TRAIN_STAGE: 1
in myloss function: summation 49.0, RMSE: 8.099823631236847, TRAIN_STAGE: 1
in myloss function: summation 168.0, RMSE: 10.159443179342345, TRAIN_STAGE: 1
in myloss function: summation 71.0, RMSE: 9.678769107100935, TRAIN_STAGE: 1
in myloss function: summation 135.0, RMSE: 10.083579296488496, TRAIN_STAGE: 1
INFO:tensorflow:Assets written to: /content/Saved models/SIA_model_v20/assets
Model Saved
Epoch 000: Loss: 92.119, Accuracy: 135.769%, TRAIN_STAGE: 1
in myloss function: summation 129.0, RMSE: 8.886185104660251, TRAIN_STAGE: 1
in myloss function: summation 140.0, RMSE: 8.430387213611755, TRAIN_STAGE: 1
in myloss function: summation 108.0, RMSE: 8.128433340529316, TRAIN_STAGE: 1
in myloss function: summation 81.0, RMSE: 10.273057689204043, TRAIN_STAGE: 1
in myloss function: summation 212.0, RMSE: 14.066170155985704, TRAIN_STAGE: 1
Epoch 001: Loss: 100.280, Accuracy: 71.033%, TRAIN_STAGE: 1
i

KeyboardInterrupt: ignored

In [0]:
print(model.trainable_variables)

In [0]:
#Load the model
# load_model = tf.keras.models.load_model('/content/Saved models/SIA_model_v12')
# load_model.trainable_variables

In [0]:
def plot_x_train(x_train):    
    fig = go.Figure() 
    x = np.arange(x_train[1].shape[1])
    for i in range(len(x_train[1])):
        fig.add_trace(go.Scatter(x=x, y=np.reshape(x_train[0][i][:],28),name='0'))
        fig.add_trace(go.Scatter(x=x, y=np.reshape(x_train[1][i][:],28),name='1'))
    fig.show()
def plot_batch(batch):    
    fig = go.Figure()
    x = np.arange(batch.shape[1])
    fig.add_trace(go.Scatter(x=x, y=np.reshape(batch, 28),name='batch'))
    fig.show()

In [0]:
def last_batch_from_train_data(series, size=DAYS_PRED, x=0):      
    
    left_index=series.shape[0]-size   
    right_index=series.shape[0]    
       
    batch_1=series.iloc[left_index:right_index,1]
    batch_2=series.iloc[left_index:right_index,1]    
    
    batch_1 = tf.transpose(np.expand_dims(batch_1,axis=0))
    batch_2 = tf.transpose(np.expand_dims(batch_2,axis=0))
    batch_1 = np.expand_dims(batch_1,axis=0)
    batch_2 = np.expand_dims(batch_2,axis=0) 
    batch_2 = np.roll(batch_2,-1)
    
    return batch_1, batch_2
batch_1, batch_2 = last_batch_from_train_data(train_data)
batch_1 = np.copy(batch_1)
batch_2 = np.copy(batch_2)

# print(batch_1,batch_2)
# batch_2[0,27,0]=2
# print(model([batch_1.astype(float), batch_2.astype(float)]).numpy())

In [195]:
list_activation_functions=['elu','exponential','hard_sigmoid','linear', 'relu', \
                           'selu', 'sigmoid', 'softmax', 'softplus' , 'softsign', 'tanh']

#Load the model
# model=create_model_v3()
if not 'G_model' in locals():
  model = tf.keras.models.load_model('/content/Saved models/SIA_model_v20')
model=G_model
# # list_of_num_filters=[2,3,5,10,20,30,50,100,1000]
# list_of_num_filters=[10]
# list_of_kernel_size=list(range(1,8))
# for kernel_size_i in list_of_kernel_size:
#   for num_filters_i in list_of_num_filters:
#     NUM_FILTERS_IN_CONV_LAYER=num_filters_i
#     KERNEL_SIZE=kernel_size_i 
#     ACTIVATION_FUNCTION = 'selu'



# One test view on model working
batch_test_1, batch_test_2, batch_true, df_batch_2 = test_batch_ri_from_train_data(train_data, r_i=600)

noise_batch_test_1=noise_detection(batch_test_1)
noise_batch_test_2=noise_detection(batch_test_2)

prediction=f28_days_prediction(model,noise_batch_test_1,noise_batch_test_2,df_batch_2)+tf.reduce_mean(batch_test_1).numpy()
# prediction=np.zeros([28,])+tf.reduce_mean(batch_test_1).numpy()
for i in range(len(prediction)):
  if (prediction[i])<0: prediction[i]=0

fig = go.Figure()
x = np.arange(len(prediction))
fig.add_trace(go.Scatter(x=x, y=np.reshape(batch_test_1,28), name ='pre_interval'))
x = np.arange(len(prediction),len(prediction)+DAYS_PRED)
fig.add_trace(go.Scatter(x=x, y=prediction, name ='prediction'))
fig.add_trace(go.Scatter(x=x, y=np.reshape(batch_true,28), name ='true'))

fig.update_layout(
    title={
        'text': f"activation function is {ACTIVATION_FUNCTION}, Num Filters is {NUM_FILTERS_IN_CONV_LAYER}, \
        Kernel size : {KERNEL_SIZE}",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()


print(f"model prediction: \n \
    {rmsse(np.reshape(batch_true,28),prediction)}")

print(f"ones line prediction: \n \
    {rmsse(np.reshape(batch_true,28),np.ones(28))}")

print(f" sum on true: {np.sum(np.reshape(batch_true,28))}, sum on prediction: {np.sum(prediction)}")

model prediction: 
     8.340948901139999
ones line prediction: 
     2.2038926600773587
 sum on true: 58, sum on prediction: 198.0


In [0]:
#Load the model
if not 'model' in locals():
  model = tf.keras.models.load_model('/content/Saved models/SIA_model_v20')

test_summation=[]
test_rmse=[]

for _ in range(5):
    summation, rmse = validation(model,display=True)
    test_summation.append(summation)
    test_rmse.append(rmse)
print(f"test summation: {sum(test_summation)}, test rmse: {sum(test_rmse)}")